In [1]:
from mysecrets import ALPACA_API_KEY_LIVE, ALPACA_SECRET_KEY_LIVE
from mysecrets import ALPACA_API_KEY_PAPER, ALPACA_SECRET_KEY_PAPER
import math as m
import numpy as np
import pandas as pd
import pandas_ta as ta
from pandas_ta.statistics import stdev
from datetime import datetime
import requests as rq
import yfinance as yf
from email.message import EmailMessage
import ssl
import smtplib
import pytz
from mysecrets import GMAIL_USER, GMAIL_PASS

# https://api.alpaca.markets
# https://paper-api.alpaca.markets
headers = {
    'accept': 'application/json',
    'APCA-API-KEY-ID': ALPACA_API_KEY_PAPER,
    'APCA-API-SECRET-KEY': ALPACA_SECRET_KEY_PAPER
}

headersLive = {
    'accept': 'application/json',
    'APCA-API-KEY-ID': ALPACA_API_KEY_LIVE,
    'APCA-API-SECRET-KEY': ALPACA_SECRET_KEY_LIVE
}

## Get Account Info

In [7]:
paperurl = 'https://paper-api.alpaca.markets/v2/account'
url = 'https://api.alpaca.markets/v2/account'


accountResponse = rq.get(paperurl, headers=headers).json()
#reqp = requests.get(url, headers=headers)
#print(response.text)
#reqp.text

accountResponse
daytradeCount = int(accountResponse['daytrade_count'])

daytradeCount


1

## View Orders

In [8]:
## Checking order status
url = 'https://api.alpaca.markets/v2/orders?status=open'

response = rq.get(url, headers=headersLive).json()
print(len(response))
# if len(response) > 0:
#     response = response[0]

response

0


[]

## View Positions

In [9]:
url = 'https://api.alpaca.markets/v2/positions'

response = rq.get(url, headers=headersLive).json()

#print(response.text)
print(len(response))

# response = response[0]
response

1


[{'asset_id': '64bbff51-59d6-4b3c-9351-13ad85e3c752',
  'symbol': 'BTCUSD',
  'exchange': 'CRYPTO',
  'asset_class': 'crypto',
  'asset_marginable': False,
  'qty': '0.006721638',
  'avg_entry_price': '64016.25',
  'side': 'long',
  'market_value': '627.75582',
  'cost_basis': '430.294059',
  'unrealized_pl': '197.461761',
  'unrealized_plpc': '0.4588995754645081',
  'unrealized_intraday_pl': '-10.293899',
  'unrealized_intraday_plpc': '-0.016133380665277',
  'current_price': '93393.28',
  'lastday_price': '94924.737',
  'change_today': '-0.0161333815441596',
  'qty_available': '0.006721638'}]

## Place Order
Look at the one below pref

In [7]:
# This is an outdated postOrder funct. in the new one, 'cash' is an input
def postOrder(signal, TP, SL, price, symbol):
    '''
    postOrder(signal, TP, SL, price, symbol)
    '''
    headers = {
        'accept': 'application/json',
        'content-type': 'application/json',
        'APCA-API-KEY-ID': ALPACA_API_KEY_PAPER,
        'APCA-API-SECRET-KEY': ALPACA_SECRET_KEY_PAPER
    }

    orderUrl = 'https://paper-api.alpaca.markets/v2/orders'
    accountUrl = 'https://paper-api.alpaca.markets/v2/account'

    accountResponse = rq.get(accountUrl, headers=headers)
    if accountResponse.status_code == 200: # successfully retrieves acct data
        account = accountResponse.json()
        orderSize = int(float(account['cash']) * .95) - 1
        print('order size: $',orderSize)
        qty = int(orderSize / price) # int(orderSize / price)
        print('qty of shares:', qty)

        if signal == 1:
            side = 'buy'
            stopPrice = price # * 1.02 may want to add this later for additional safety, but it kinda cuts into profits
        elif signal == -1:
            side = 'sell'
            stopPrice = price # * .98
            
        # payload = {
        #     'symbol': symbol,
        #     'qty': qty,
        #     'side': side,
        #     #'notional': orderSize, #can't do notional with a stop order type
        #     'type': 'market',#'stop'
        #     'time_in_force': 'day',  # day orders placed after close are submitted the next day, or can be filled in after hours
        #    # 'stop_price': stopPrice,
        #     'order_class': 'bracket',
        #     'take_profit': {'limit_price': TP},
        #     'stop_loss': {'stop_price': SL}
        # }
        payload = {
            'symbol': symbol,
            'qty': qty,
            'side': side,
            #'notional': orderSize, #can't do notional with a stop order type
            'type': 'stop', # 'market',
            'time_in_force': 'gtc',  # orders are good until cancelled
            'stop_price': stopPrice,
            'order_class': 'bracket',
            'take_profit': {'limit_price': TP},
            'stop_loss': {'stop_price': SL}
        }

        orderResponse = rq.post(orderUrl, json=payload, headers=headers) # this line posts the order (EVERYTIME IT RUNS)
        if orderResponse.status_code == 200:
            print('order succesfully placed')
            # log order successfuly placed
            # slack order succesffully placed
        else:
            print('order failed to place', orderResponse.status_code)
            #log order did not succcesffuly place
            # slack it too 

    else:
        print('log FAILED to receive acct data')


########
signal = 1 #
TP = 104
SL = 100.5
symbol = 'XOM'
price = 101.7
postOrder(signal, TP, SL, price, symbol)

order size: $ 97422
qty of shares: 957
order succesfully placed


## Updated make order 
this works more like the live one

In [3]:
def checkPositions():
    '''
    Checks Alpaca positions and open orders
    positionNum, orderNum, positions, orders, portfolioVal, cash = checkPositions()
    Inputs:
    none
    Outputs:
    positions & orders - json files including info
    positionNum & orderNum - number of positions and orders (int)
    '''
    headers = {
        "accept": "application/json",
        "APCA-API-KEY-ID": ALPACA_API_KEY_LIVE,
        "APCA-API-SECRET-KEY": ALPACA_SECRET_KEY_LIVE
    }

    accountUrl = 'https://api.alpaca.markets/v2/account'
    positionsUrl = "https://api.alpaca.markets/v2/positions"
    ordersUrl = "https://api.alpaca.markets/v2/orders?status=open"

    accountResponse = rq.get(accountUrl, headers=headers)
    positionsResponse = rq.get(positionsUrl, headers=headers)
    ordersResponse = rq.get(ordersUrl, headers=headers)

    if positionsResponse.status_code == 200 & ordersResponse.status_code == 200 & accountResponse.status_code == 200:
        positions = positionsResponse.json() # converts to json format
        orders = ordersResponse.json()
        account = accountResponse.json()
        positionNum = len(positions) # number of positions (0 if none)
        orderNum = len(orders)
        portfolioVal = float(account['portfolio_value'])
        cash = float(account['cash'])
        if positionNum > 0:
            positions = positions[0] # This will make subfunct only return info on the first order
        if orderNum > 0:
            orders = orders[0]
    return positionNum, orderNum, positions, orders, portfolioVal, cash
    
def postOrder(signal, SL, TP, price, symbol, cash, portfolioVal, latestTime):
    '''
    Post order to Alpaca through requests (not through their API)
    postOrder(signal, SL, TP, price, symbol)
    '''
    headers = {
        'accept': 'application/json',
        'content-type': 'application/json',
        'APCA-API-KEY-ID': ALPACA_API_KEY_LIVE,
        'APCA-API-SECRET-KEY': ALPACA_SECRET_KEY_LIVE
    }

    ordersUrl = 'https://api.alpaca.markets/v2/orders'
    
    orderSize = int(cash * .98 - 1)
    qty = int(orderSize / price)

    if signal == 1:
        side = 'buy'
        stopPrice = price # * +.10 # add 10 cents; may want to add this later for additional safety, but it kinda cuts into profits
    elif signal == -1:
        side = 'sell'
        stopPrice = price # * .98 # - .10
    
    payload = {
        'symbol': symbol,
        'qty': qty,
        'side': side,
        #'notional': orderSize, #can't do notional with a stop order type
        'type': 'market', # 'market',
        'time_in_force': 'gtc',  # orders are good until cancelled
        #'stop_price': stopPrice,
        'order_class': 'bracket',
        'take_profit': {'limit_price': TP},
        'stop_loss': {'stop_price': SL}
    }
    orderResponse = rq.post(ordersUrl, json=payload, headers=headers) # Attempt to submit order
    print(orderResponse.status_code)
    if orderResponse.status_code == 200:
        recordResults('order success', portfolioVal, latestTime, symbol, qty, signal, stopPrice, SL, TP) # record order succeeded
    else:
        recordResults('order failed', portfolioVal, latestTime, symbol) # record order failed

def recordResults(status, portfolioVal, latestTime=None, symbol=None, qty=None, signal=None, price=None, SL=None, TP=None, unrealizedPL=None,unrealizedPLPC=None):
    '''
    Takes status of program end, creates a subject and body, and logs it as well as sending out email
    recordResults(status, symbol=None, qty=None, signal=None, price=None, SL=None, TP=None, unrealizedPL=None)
    '''
    currentTime = datetime.now(pytz.timezone('US/Eastern')) # get the local time of stock exchange
    currentTime = currentTime.strftime('%b %d, %Y %H:%M EST')

    side = 'long' if signal == 1 else 'short'
    if status == 'order success':
        subject = f'{currentTime}: Order placed on {symbol}'
        body = f'Order placed on {symbol} for {side} position at ${price}\n'
        body += f'{qty} shares totalling order size of ${qty*price: .2f}\n'
        body += f'Stop loss set at ${SL} | TP set at ${TP}\n'
        body += f'Data evaluated using latest data from {latestTime} EST.\n'
    elif status == 'order failed':
        subject = f'{currentTime}: Order failed to place on {symbol}!'
        body = 'sorry mate I tried. well you tried a while ago. but i slipped thru haha\n'
        body += f'Data evaluated using latest data from {latestTime} EST.\n'
    # elif status == 'data retrieval failed':
    #     subject = f'{currentTime}: Account data retreival failed!'
    #     body = 'bro this is kind of a let down I thought you could do better than this.\n'
    elif status == 'no orders made':
        subject = f'{currentTime}: No signals, no orders made'
        body = 'No signals, no orders made, yea\n'
        body += f'Data evaluated using latest data from {latestTime} EST.\n'
    elif status == 'positions already':
        subject = f'{currentTime}: Already holding a position of {symbol} at ${price: .2f}'
        body = f'Unrealized P/L: ${unrealizedPL: .2f} ({unrealizedPLPC: .2f}%)\n'
    elif status == 'order and no position':
        subject = f'{currentTime}: Order for {symbol} still open at ${price}'
        body = f'Order still open for {qty} shares totalling order size of ${qty*price: .2f}\n'
    else:
        subject = 'PROBLEM PROBLEM'
        body = 'BIG PROBLEM HAPPENED'
    body += f'Current portfolio value: ${portfolioVal: .2f}'
    sendEmail(subject, body)


def sendEmail(subject, body):
    '''
    Sends email to self using user in mysecrets file containing provided info
    sendEmail(subject, body)
    '''
    GMAIL_RECEIVER = GMAIL_USER # set email recipient as self
    em = EmailMessage()
    em['From'] = GMAIL_USER
    em['To'] = GMAIL_USER
    em['subject'] = subject
    em.set_content(body)

    context = ssl.create_default_context()
    with smtplib.SMTP_SSL('smtp.gmail.com', 465, context=context) as smtp:
        smtp.login(GMAIL_USER, GMAIL_PASS)
        smtp.sendmail(GMAIL_USER, GMAIL_RECEIVER, em.as_string())
    
    
positionNum, orderNum, positions, orders, portfolioVal, cash = checkPositions()
#def postOrder(signal, SL, TP, price, symbol, cash, portfolioVal, latestTime):
postOrder(1, 33.77, 34.97, 34.21, 'EXC', cash, portfolioVal, 'I ACUTALLY JUST DID MARKET OREDER') # create buy order

200


## Close Orders
- always close orders first, closing positions is an order that needs to not get cancelled

In [10]:
ordersUrl = 'https://paper-api.alpaca.markets/v2/orders?status=open' #close orders before positions, or it'll cancel close position ORDER
closeAllOrders = rq.delete(ordersUrl, headers=headers) # attempt to delete all orders

print('status code:',closeAllOrders.status_code)
closeAllOrders.json()

status code: 207


[{'id': '5be26df9-86fe-4362-88e6-40ae3d2817a4', 'status': 200}]

## Close Positions

In [3]:
positionsUrl = 'https://api.alpaca.markets/v2/positions'
closeAllPositions = rq.delete(positionsUrl, headers=headers)

print('status code:',closeAllPositions.status_code)
closeAllPositions.json()

status code: 401


{'code': 40110000, 'message': 'request is not authorized'}

## Cancel certain order

In [ ]:
orderID= '234234234'
url = f"https://paper-api.alpaca.markets/v2/orders/{orderID}"
response = rq.delete(url, headers=headers)

print('status code:',response.status_code)
response.json()

## Close certain position

In [33]:
symbol = 'XOM'
url = f"https://paper-api.alpaca.markets/v2/positions/{symbol}?percentage=100"
response = rq.delete(url, headers=headers)

print('status code:',response.status_code)
response.json()

status code: 200


{'id': 'dc780516-0dad-4919-8635-ca17d74def33',
 'client_order_id': '4ccb5311-8b7a-48ad-a3cb-26bd7481a519',
 'created_at': '2023-12-22T18:54:47.669619396Z',
 'updated_at': '2023-12-22T18:54:47.669685456Z',
 'submitted_at': '2023-12-22T18:54:47.669110466Z',
 'filled_at': None,
 'expired_at': None,
 'canceled_at': None,
 'failed_at': None,
 'replaced_at': None,
 'replaced_by': None,
 'replaces': None,
 'asset_id': '092efc51-b66b-4355-8132-d9c3796b9a76',
 'symbol': 'XOM',
 'asset_class': 'us_equity',
 'notional': None,
 'qty': '43',
 'filled_qty': '0',
 'filled_avg_price': None,
 'order_class': '',
 'order_type': 'market',
 'type': 'market',
 'side': 'buy',
 'time_in_force': 'day',
 'limit_price': None,
 'stop_price': None,
 'status': 'pending_new',
 'extended_hours': False,
 'legs': None,
 'trail_percent': None,
 'trail_price': None,
 'hwm': None,
 'subtag': None,
 'source': None}

# Options

In [5]:
# https://www.youtube.com/watch?v=B0Z7oCmr5nM
# based from 2:01 into this video. the start is in follow_alpaca_options_tut.py

from datetime import datetime, timedelta

import alpaca
from alpaca.data.historical.option import OptionHistoricalDataClient, OptionLatestQuoteRequest
from alpaca.data.historical.stock import StockHistoricalDataClient, StockLatestTradeRequest
from alpaca.trading.client import TradingClient, GetAssetsRequest
from alpaca.trading.requests import GetOptionContractsRequest, LimitOrderRequest, MarketOrderRequest, GetOrdersRequest
from alpaca.trading.enums import AssetStatus, ContractType, OrderSide, OrderType, TimeInForce, QueryOrderStatus


# from alpaca.trading.stream import TradingStream
trade_client = TradingClient(api_key=ALPACA_API_KEY_PAPER, secret_key=ALPACA_SECRET_KEY_PAPER, paper=True)
stock_data_client = StockHistoricalDataClient(api_key=ALPACA_API_KEY_PAPER, secret_key=ALPACA_SECRET_KEY_PAPER)
option_data_client = OptionHistoricalDataClient(api_key=ALPACA_API_KEY_PAPER, secret_key=ALPACA_SECRET_KEY_PAPER)

In [3]:
acct = trade_client.get_account()

print(f"options approved level {acct.options_approved_level}")
print(f"options trading level: {acct.options_trading_level}")
print(f"options buyuing power: ${acct.options_buying_power}")

options approved level 2
options trading level: 2
options buyuing power: $30034.74


### change max allowed options trading level

In [15]:
acct_config = trade_client.get_account_configurations()
print(f"max options trading level: {acct_config.max_options_trading_level}")

acct_config.max_options_trading_level = 2
trade_client.set_account_configurations(acct_config)

print(f"max options trading level: {acct_config.max_options_trading_level}")


max options trading level: 1
max options trading level: 2


In [23]:
req = GetAssetsRequest(
    status=AssetStatus.ACTIVE,
    attributes="has_options"
    # attributes="options_enabled" # what he did in video has been changed
)

options_enabled_underlyings = trade_client.get_all_assets(req)
print(f"number of underlyings with options: {len(options_enabled_underlyings)}")

underlying_symbol = "TSLA"
underlying = trade_client.get_asset(symbol_or_asset_id=underlying_symbol)
print(f"{underlying_symbol} has options? {'has_options' in underlying.attributes}")
# print(options_enabled_underlyings)

number of underlyings with options: 5769
TSLA has options? True


### query positions in acct

In [42]:
positions = trade_client.get_all_positions()

for p in positions:
    print(p)

asset_id=UUID('8ccae427-5dd0-45b3-b5fe-7ba5e422c766') symbol='TSLA' exchange=<AssetExchange.NASDAQ: 'NASDAQ'> asset_class=<AssetClass.US_EQUITY: 'us_equity'> asset_marginable=True avg_entry_price='422.9697' qty='100' side=<PositionSide.LONG: 'long'> market_value='41209.85' cost_basis='42296.97' unrealized_pl='-1087.12' unrealized_plpc='-0.0257020774774174' unrealized_intraday_pl='-1087.12' unrealized_intraday_plpc='-0.0257020774774174' current_price='412.0985' lastday_price='431.66' change_today='-0.0453169160913682' swap_rate=None avg_entry_swap_rate=None usd=None qty_available='0'


### covered call

In [43]:
today = datetime.now().date()
min_expiration = today + timedelta(days=14)
max_expiration = today + timedelta(days=98)

underlying_trade_request = StockLatestTradeRequest(symbol_or_symbols=underlying_symbol)
underlying_latest_trade = stock_data_client.get_stock_latest_trade(underlying_trade_request)
underlying_last_price = underlying_latest_trade[underlying_symbol].price

print(f"{underlying_symbol} last trade price: {underlying_last_price}")

min_strike = str(round(underlying_last_price*1.02,2))
print(f"Min expr: {min_expiration}, Max expr: {max_expiration}, min strike {min_strike}")

req = GetOptionContractsRequest(
    underlying_symbols=[underlying_symbol], # specify symbol(s)
    status=AssetStatus.ACTIVE, # specify asset status: active (default)
    expiration_date=None, # specify expr date (specified date + 1 day range)
    expiration_date_gte=min_expiration, # can pass date obj or string (YYYY-MM-DD)
    expiration_date_lte=max_expiration,
    root_symbol=underlying_symbol, # specify root symbol
    type=ContractType.CALL, # either call or put
    style=None, # either american or european
    strike_price_gte=min_strike, # strike price range
    strike_price_lte=None,
    limit=None, # specifiy limit
    page_token=None
)

res = trade_client.get_option_contracts(req)
options_chain_list = res.option_contracts

if res.next_page_token is not None: # if its multiple pages
    req = GetOptionContractsRequest(
        underlying_symbols=[underlying_symbol], # specify symbol(s)
        status=AssetStatus.ACTIVE, # specify asset status: active (default)
        expiration_date=None, # specify expr date (specified date + 1 day range)
        expiration_date_gte=min_expiration, # can pass date obj or string (YYYY-MM-DD)
        expiration_date_lte=max_expiration,
        root_symbol=underlying_symbol, # specify root symbol
        type=ContractType.CALL, # either call or put
        style=None, # either american or european
        strike_price_gte=min_strike, # strike price range
        strike_price_lte=None,
        limit=None, # specifiy limit
        page_token=res.next_page_token
    )
res = trade_client.get_option_contracts(req)
options_chain_list.extend(res.option_contracts)

print(f"number of options retreived: {len(options_chain_list)}")

options_chain_list


TSLA last trade price: 403.74
Min expr: 2025-01-15, Max expr: 2025-04-09, min strike 411.81
number of options retreived: 200


[{   'close_price': '18.13',
     'close_price_date': datetime.date(2024, 12, 31),
     'expiration_date': datetime.date(2025, 1, 17),
     'id': '390b0617-6568-4600-b3aa-46f3085104e3',
     'name': 'TSLA Jan 17 2025 412.5 Call',
     'open_interest': None,
     'open_interest_date': None,
     'root_symbol': 'TSLA',
     'size': '100',
     'status': <AssetStatus.ACTIVE: 'active'>,
     'strike_price': 412.5,
     'style': <ExerciseStyle.AMERICAN: 'american'>,
     'symbol': 'TSLA250117C00412500',
     'tradable': True,
     'type': <ContractType.CALL: 'call'>,
     'underlying_asset_id': UUID('8ccae427-5dd0-45b3-b5fe-7ba5e422c766'),
     'underlying_symbol': 'TSLA'},
 {   'close_price': '17.11',
     'close_price_date': datetime.date(2024, 12, 31),
     'expiration_date': datetime.date(2025, 1, 17),
     'id': '04d03ea6-041f-4b3f-b58b-c5c6268f5e8f',
     'name': 'TSLA Jan 17 2025 415 Call',
     'open_interest': '2639',
     'open_interest_date': datetime.date(2024, 12, 30),
     'ro

### pick an option

In [39]:
highest_yield = 0
highest_yield_option = options_chain_list[0]

for o in options_chain_list:
    option_symbol = o.symbol
    option_strike = o.strike_price
    option_expr = o.expiration_date
    option_quote_request = OptionLatestQuoteRequest(symbol_or_symbols=option_symbol)
    option_quote = option_data_client.get_option_latest_quote(request_params=option_quote_request)
    option_quote_bid = option_quote[option_symbol].bid_price
    option_quote_ask = option_quote[option_symbol].ask_price
    if option_quote_bid > 0 and option_quote_bid < option_quote_ask:
        DTE = (option_expr-today).days
        overwrite_yield = option_quote_bid / underlying_last_price / DTE *365
        if overwrite_yield > highest_yield:
            highest_yield = overwrite_yield
            highest_yield_option=o
            highest_yield_option_bid = option_quote_bid

print(f"highest yield option: {highest_yield_option}")
print(f"highest yield: {highest_yield}")
print(f"highest yield bid: {highest_yield_option_bid}")

highest yield option: id='abe0fafb-9e6f-448e-8e0a-020f3f0824d8' symbol='TSLA250117C00430000' name='TSLA Jan 17 2025 430 Call' status=<AssetStatus.ACTIVE: 'active'> tradable=True expiration_date=datetime.date(2025, 1, 17) root_symbol='TSLA' underlying_symbol='TSLA' underlying_asset_id=UUID('8ccae427-5dd0-45b3-b5fe-7ba5e422c766') type=<ContractType.CALL: 'call'> style=<ExerciseStyle.AMERICAN: 'american'> strike_price=430.0 size='100' open_interest='8332' open_interest_date=datetime.date(2024, 12, 26) close_price='26.85' close_price_date=datetime.date(2024, 12, 27)
highest yield: 0.8369570383043438
highest yield bid: 17.37


### Sell that option (Place the order)
given that you hold 100 of the underlying (to make a covered call)

In [40]:
req = LimitOrderRequest(
    symbol=highest_yield_option.symbol,
    qty=1,
    limit_price = highest_yield_option_bid,
    side=OrderSide.SELL,
    type=OrderType.LIMIT,
    time_in_force = TimeInForce.DAY
)

res = trade_client.submit_order(req)
res

{   'asset_class': <AssetClass.US_OPTION: 'us_option'>,
    'asset_id': UUID('abe0fafb-9e6f-448e-8e0a-020f3f0824d8'),
    'canceled_at': None,
    'client_order_id': '0f4d5bc8-90af-427d-8a72-de44c2367028',
    'created_at': datetime.datetime(2024, 12, 30, 21, 0, 47, 959025, tzinfo=TzInfo(UTC)),
    'expired_at': None,
    'extended_hours': False,
    'failed_at': None,
    'filled_at': None,
    'filled_avg_price': None,
    'filled_qty': '0',
    'hwm': None,
    'id': UUID('9f77bdd4-1726-4b2a-89ad-317d96ca8729'),
    'legs': None,
    'limit_price': '17.37',
    'notional': None,
    'order_class': <OrderClass.SIMPLE: 'simple'>,
    'order_type': <OrderType.LIMIT: 'limit'>,
    'position_intent': <PositionIntent.SELL_TO_OPEN: 'sell_to_open'>,
    'qty': '1',
    'replaced_at': None,
    'replaced_by': None,
    'replaces': None,
    'side': <OrderSide.SELL: 'sell'>,
    'status': <OrderStatus.ACCEPTED: 'accepted'>,
    'stop_price': None,
    'submitted_at': datetime.datetime(2024, 1

In [41]:
req = GetOrdersRequest(
    status=QueryOrderStatus.ALL,
    symbols=None, # [highest_yield_option.symbol]
    limit=2
)

orders = trade_client.get_orders(req)

orders

[{   'asset_class': <AssetClass.US_OPTION: 'us_option'>,
     'asset_id': UUID('abe0fafb-9e6f-448e-8e0a-020f3f0824d8'),
     'canceled_at': None,
     'client_order_id': '0f4d5bc8-90af-427d-8a72-de44c2367028',
     'created_at': datetime.datetime(2024, 12, 30, 21, 0, 47, 959025, tzinfo=TzInfo(UTC)),
     'expired_at': None,
     'extended_hours': False,
     'failed_at': None,
     'filled_at': None,
     'filled_avg_price': None,
     'filled_qty': '0',
     'hwm': None,
     'id': UUID('9f77bdd4-1726-4b2a-89ad-317d96ca8729'),
     'legs': None,
     'limit_price': '17.37',
     'notional': None,
     'order_class': <OrderClass.SIMPLE: 'simple'>,
     'order_type': <OrderType.LIMIT: 'limit'>,
     'position_intent': <PositionIntent.SELL_TO_OPEN: 'sell_to_open'>,
     'qty': '1',
     'replaced_at': None,
     'replaced_by': None,
     'replaces': None,
     'side': <OrderSide.SELL: 'sell'>,
     'status': <OrderStatus.ACCEPTED: 'accepted'>,
     'stop_price': None,
     'submitted_at'

### Cancel order

In [ ]:
trade_client.cancel_order_by_id(order_id=orders[0].id)

### Market order

In [ ]:
req = MarketOrderRequest(
    symbol=highest_yield_option.symbol,
    qty=1,
    limit_price = highest_yield_option_bid,
    side=OrderSide.SELL,
    type=OrderType.MARKET,
    time_in_force = TimeInForce.DAY
)

res = trade_client.submit_order(req)
res

### Close position

In [ ]:
close_position_symbol = positions[0].symbol
trade_client.close_position(
    symbol_or_asset_id=close_position_symbol
)

### exercise options

In [ ]:
close_position_symbol = positions[0].symbol
trade_client.exercise_options_position(
    symbol_or_asset_id=close_position_symbol
)

# Other stuff

In [10]:
'''
    print(ordersResponse.status_code)
    print(ordersResponse.text)
    print(type(positionsResponse.status_code))  # status code should be 200
    print(type(positionsResponse))
    print(len(positionsResponse.json()))   # number of positions
    print(positionsResponse.json()[0]['symbol'])  # how to reach a certain value
'''

#     'symbol', 'side' 'market_value' 'avg_entry_price'

# return orderstatus,positionstatus, stockname, allat

def checkPositions():
    '''
    positions, orders, positionNum, orderNum = checkPositions()
    '''
    headers = {
        'accept': 'application/json',
        'APCA-API-KEY-ID': API_KEY,
        'APCA-API-SECRET-KEY': SECRET_KEY
    }

    positionsUrl = 'https://paper-api.alpaca.markets/v2/positions'
    ordersUrl = 'https://paper-api.alpaca.markets/v2/orders?status=open'

    positionsResponse = rq.get(positionsUrl, headers=headers)
    ordersResponse = rq.get(ordersUrl, headers=headers)

    if positionsResponse.status_code == 200 & ordersResponse.status_code == 200:
        positions = positionsResponse.json() # converts to json format
        orders = ordersResponse.json()
        positionNum = len(positions) # number of position (0 if none)
        orderNum = len(orders)
        #if len(positions) != 0 & len(orders) != 0: # if there are both position(s) and order(s)
        #    closeAllOrders = rq.delete(positionsUrl, headers=headers) # attempt to delete all orders
        #    closeAllPositions = rq.delete(ordersUrl, headers=headers)
        #  BOTH POSITIONS AND ORDERS OPEN< BUT THATS RIGHT
        print('succesfuly got acct info')
    else:
        print('failed to get acct info')

    return positions, orders, positionNum, orderNum

positions, orders, positionNum, orderNum = checkPositions()
print('positionNum:',positionNum, '\nOrderNum:',orderNum)
print(positions)
# if len(positions) > 1:
#     positions = positions[0]

# positions

succesfuly got acct info
positionNum: 0 
OrderNum: 0
[]


In [12]:
round(2.32453543,2)

2.32